In [1]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics

import warnings
warnings.simplefilter('ignore')

In [2]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    """
    The class provides basic functionality for retrieving
    a subset of columns from the dataset.
    """
    
    def __init__(self, feature_names):
        """
        Initialize class instance by setting
        a list of columns to retrieve from the dataset.
        """
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
        self.feature_names = feature_names
        
    def fit(self, X, y=None):
        """
        Fit FeatureSelector to X, but really do nothing.
        Return self.
        """
        return self
    
    def transform(self, X, y=None):
        """
        Transform X using feature selection. 
        Return column-subset of X.
        """
        return X[self.feature_names]

In [3]:
class FeatureGenerator(BaseEstimator, TransformerMixin):
    """
    Required columns: table. X is DataFrame.
    """
    
    features = {
             'ABS (антиблокировочная система)',
             'AUX/iPod',
             'Bluetooth',
             'CD/MP3 проигрыватель',
             'ESP (система поддержания динамической стабильности)',
             'USB',
             'Автозапуск двигателя',
             'Антипробуксовочная система',
             'Датчик дождя',
             'Иммобилайзер',
             'Камера заднего вида',
             'Климат-контроль',
             'Кондиционер',
             'Контроль мертвых зон на зеркалах',
             'Круиз-контроль',
             'Ксеноновые фары',
             'Легкосплавные диски',
             'Люк',
             'Материал салона - натуральная кожа',
             'Мультимедийный экран',
             'Обогрев зеркал',
             'Обогрев лобового стекла',
             'Обогрев руля',
             'Обогрев сидений',
             'Панорамная крыша',
             'Парктроники',
             'Подушки безопасности боковые',
             'Подушки безопасности задние',
             'Подушки безопасности передние',
             'Противотуманные фары',
             'Рейлинги на крыше',
             'Светодиодные фары',
             'Сигнализация',
             'Системы помощи',
             'Управление мультимедиа с руля',
             'Фаркоп',
             'Штатная навигация',
             'Электрорегулировка сидений',
             'Электростеклоподъемники задние',
             'Электростеклоподъемники передние'}
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        self.add_features(X)
        return X[self.features].values
    
    def add_features(self, X):
        for imp in self.features:
            X[imp] = X.table.apply(lambda x: int(imp in x)).astype('int8')

In [4]:
class ToIntTransformer(BaseEstimator, TransformerMixin):
    """
    Required columns: volume, show, run, pages, update, year.
    """
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        # self.to_int(X, 'cost', ' ')
        self.to_int(X, 'volume', ' см3')
        self.fix_show(X)
        self.fix_run(X)
        self.fix_dates(X)
        self.add_restyle(X)
        self.add_upd_flags(X)
        self.year_to_old(X)
        return X.drop(['model', 'update', 'pages', 'year'], axis=1).values
    
    def to_int(self, X, column_name, phrase):
        X[column_name] = X[column_name].str.replace(phrase, '').astype('int32')
        
    def year_to_old(self, X):
        X['age'] = (2019 - X['year']).astype('int8')
        
    def fix_show(self, X):
        X['today_views'] = X['show'].str.extract('\+(.+) ')
        X['show'] = X['show'].str.extract('(.*)' + ' '*25)
        X.rename(columns={'show': 'all_views'}, inplace=True)
        today_view_mask = pd.isna(X['today_views'])
        X.loc[today_view_mask, 'today_views'] = X[today_view_mask]['all_views']
        X['today_views'] = X['today_views'].astype('int')
        X['all_views'] = X['all_views'].astype('int')
        
    def fix_run(self, X):
        X['run'] = X['run'].str.replace(' км', '')
        miles_mask = X['run'].str.endswith(' миль')
        X.loc[miles_mask, 'run'] = X[miles_mask]['run'].str.replace(' миль', '').astype('int') * 1.60934
        X['run'] = X['run'].astype('int')
        
    def add_restyle(self, X):
        self.create_model(X)
        X['is_restyle'] = X['model'].str.endswith('(рестайлинг)').astype('int8')
        
    def add_upd_flags(self, X):
        X['modified'] = X['update'].apply(
            lambda x: int(not x.split()[0] == 'Опубликовано')).astype('int8')
        X['up'] = X['update'].apply(
            lambda x: int(len(x.split()) == 4)).astype('int8')
    
    def create_model(self, X):
        two_word_names = ('Alfa Romeo', 'Great Wall', 'Lada (ВАЗ)')
        two_word_names_mask = X['pages'].str.startswith(two_word_names)
        X.loc[two_word_names_mask, 'model'] = (X[two_word_names_mask]['pages'].str.split()
                                                .apply(lambda name: ' '.join(name[2:])))
        X.loc[~two_word_names_mask, 'model'] = (X[~two_word_names_mask]['pages'].str.split()
                                                 .apply(lambda name: ' '.join(name[1:])))
    
    def fix_dates(self, df):
        today=pd.Timestamp(2019, 11, 23)
        df['days_ago'] = df['update'].apply(
            lambda x: (today - pd.Timestamp(x.split()[1])).days)

In [16]:
class CatTransformer(BaseEstimator, TransformerMixin):
    """
    Required columns: cuzov, fuel, pages, region, update.
    """    
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        self.cut(X, ['cuzov', 'fuel'])
        self.fix_names(X)
        self.fix_region(X)
        return X.drop(['update'], axis=1)
    
    def cut(self, X, column_names):
        for col in column_names:
            X[col] = X[col].apply(lambda x: x.split()[0])
        
    def fix_names(self, df):
        two_word_names = ('Alfa Romeo', 'Great Wall', 'Lada (ВАЗ)')
        two_word_names_mask = df['pages'].str.startswith(two_word_names)
        # df.loc[two_word_names_mask, 'model'] = (df[two_word_names_mask]['pages'].str.split()
        #                                         .apply(lambda name: ' '.join(name[2:])))
        # df.loc[~two_word_names_mask, 'model'] = (df[~two_word_names_mask]['pages'].str.split()
        #                                          .apply(lambda name: ' '.join(name[1:])))
        df.loc[two_word_names_mask, 'pages'] = (df[two_word_names_mask]['pages'].str.split()
                                                .apply(lambda name: ' '.join(name[:2])))
        df.loc[~two_word_names_mask, 'pages'] = (df[~two_word_names_mask]['pages'].str.split()
                                                 .apply(lambda name: ' '.join(name[:1])))
        df.rename(columns={'pages': 'brand'}, inplace=True)
        vc = df.brand.value_counts()
        df.brand = df.brand.apply(lambda x: x if vc[x] > 100 else 'other')
        
    def fix_region(self, X):
        
        def _get_region(lst):
            if len(lst) == 1:
                return lst[0]
            return lst[1]
        
        tmp = list(map(lambda s: s.split(', '), X.region))
        X.region = list(map(_get_region, tmp))


In [6]:
class ColumnTranslation:
    
    def __init__(self, column_name, to_save, default='Other'):
        self.column_name = column_name
        self.to_save = to_save
        self.default = default


class Translator(BaseEstimator, TransformerMixin):
    
    def __init__(self, translations):
        self.translations = translations[:]
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        for tr in self.translations:
            X[tr.column_name] = X[tr.column_name].apply(
                lambda x: x if x in tr.to_save else tr.default)
        return X[[tr.column_name for tr in self.translations]].values

In [31]:
avto = pd.read_csv('final.csv')

X = avto.drop(['cost'], axis=1)
y = avto.cost.apply(lambda x: int(x.replace(' ', '')))
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2, random_state=42)

feat_pipeline = Pipeline(
    steps=[
        ('feat_selector', FeatureSelector(['table'])),
        ('feat_generator', FeatureGenerator())
    ]
)

int_pipeline = Pipeline(
    steps=[
        ('int_selector', FeatureSelector(['volume', 'show', 'run', 'pages', 'update', 'year'])),
        ('int_transformer', ToIntTransformer())
    ]
)

cat_pipeline = Pipeline(
    steps=[
        ('cat_selector', FeatureSelector(['cuzov', 'fuel', 'pages', 'region', 'update'])),
        ('cat_transformer', CatTransformer()),
        ('cat_encoder', OneHotEncoder())
    ]
)

color_pipeline = Pipeline(
    steps=[
        ('color_selector', FeatureSelector(['color'])),
        ('color_translator', Translator(
            [ColumnTranslation(
                column_name='color',
                to_save=['черный', 'серебристый', 'синий', 'серый', 'белый'],
                default='другой'
            )])),
        ('color_encoder', OneHotEncoder())
    ]
)

no_proc_pipeline = Pipeline(
    steps=[
        ('no_proc_selector', FeatureSelector(['drive-unit', 'state', 'transmission'])),
        ('no_proc_imputer', SimpleImputer(strategy='most_frequent')),
        ('no_proc_encoder', OneHotEncoder())
    ]
)

pipeline = FeatureUnion(transformer_list= [
    ('feat', feat_pipeline),
    ('int', int_pipeline),
    ('cat', cat_pipeline),
    ('color', color_pipeline),
    ('no_proc', no_proc_pipeline)
])

In [32]:
import xgboost as xg

In [39]:
pip = Pipeline(
    steps=[
        ('preprocessing', pipeline),
        ('model', xg.XGBRegressor(
            max_depth=8,
            learning_rate=0.1, 
            n_estimators=200
        ))
    ]
)

pip.fit(X_train, y_train)
y_true, y_pred = y_test, pip.predict(X_test)
print(metrics.r2_score(y_true, y_pred))

[00:31:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.830933846516222
